# Notebook for objective 1
##### Xinli(Chloe) Gu

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import re

%matplotlib inline

**Load Data for obj1 (DIAG.txt, PX.txt, RX.txt)**

In [3]:
diag = pd.read_csv('Data/APLD Raw Data/DIAG.txt/DIAG.txt', 
                   dtype={"PATIENT_ID": int,"CLAIM_ID": object,"CLAIM_TYP_CD": object,"SERVICE_DATE": str,"MONTH_ID": int,
                          "DIAGNOSIS_CODE": object,"DIAG_VERS_TYP_ID": int,"DIAG_CD_POSN_NBR": int,"PROVIDER_ID": object,
                          "RESTATE_FLAG": object,"FLEXIBLE_FLD_1_CHAR": object,"FLEXIBLE_FLD_2_CHAR": object}, 
                   sep="|",
                   engine='python')
diag.head()

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR
0,999944505,10458194885000421223,P,12/12/2019,201912,C50.111,2,1,7976755,0,1588637250,NaN
1,999944505,10458181473000479341,P,09/10/2019,201909,C50.111,2,1,7976755,0,1588637250,NaN
2,999944505,10458165065000341912,P,05/20/2019,201905,C50.111,2,1,7976755,0,1588637250,NaN
3,999944505,10458161262800585500,P,05/20/2019,201905,C50.111,2,1,8117971,0,1285667204,NaN
4,999944505,10458157449200367497,P,02/21/2019,201902,C50.111,2,1,7976755,0,1588637250,NaN


In [4]:
diag.describe(include='all')

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR
count,3.192462e+06,3192462,3192462,3192462,3.192462e+06,3192462,3.192462e+06,3.192462e+06,3192462,3192462,2936942,0
unique,NaN,1704513,2,2313,NaN,1471,NaN,NaN,122722,1,125048,0
top,NaN,10655099975600350055,P,02/01/2019,NaN,C50.919,NaN,NaN,0,0,1366479099,NaN
freq,NaN,237,2081284,4884,NaN,352621,NaN,NaN,338616,3192462,8847,NaN
mean,7.892070e+08,NaN,NaN,NaN,2.017630e+05,NaN,1.900690e+00,1.869614e+00,NaN,NaN,NaN,NaN
std,4.890956e+08,NaN,NaN,NaN,1.466147e+02,NaN,2.990777e-01,1.280753e+00,NaN,NaN,NaN,NaN
min,2.938854e+07,NaN,NaN,NaN,2.013090e+05,NaN,1.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN
25%,3.500931e+08,NaN,NaN,NaN,2.017050e+05,NaN,2.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN
50%,7.396496e+08,NaN,NaN,NaN,2.018060e+05,NaN,2.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN
75%,1.188289e+09,NaN,NaN,NaN,2.019030e+05,NaN,2.000000e+00,2.000000e+00,NaN,NaN,NaN,NaN


In [5]:
len(diag)

3192462

In [6]:
px = pd.read_csv('Data/APLD Raw Data/PX.txt/PX.txt', 
                   dtype={"PATIENT_ID": int,"CLAIM_ID": object,"CLAIM_LINE_ITEM": int,
                          "CLAIM_TYP_CD": object,"PROCEDURE_CODE": object,"PRC1_MOD_CD": object,"PRC1_MOD_DESC": object,
                          "PRC_VERS_TYP_ID": object,"PROVIDER_BILLING_ID": int,"PROVIDER_FACILITY_ID": int,
                          "PROVIDER_REFERRING_ID": int,"PROVIDER_RENDERING_ID": int,"SVC_CRGD_AMT": float,
                          "SERVICE_DATE": str,"MONTH_ID": int,"UNIT_OF_SVC_AMT": float,
                          "PLACE_OF_SERVICE": object,"PAYER_PLAN_ID": object,"PAY_TYPE": object,"NDC": object,
                          "PRODUCT": object,"DIAGNOSIS_CODE": object,"DIAG_CD_POSN_NBR": int,"DIAG_VERS_TYP_ID": int,
                          "DIAG_DESC": object,"WEEK_END_FRI": object,
                          "RESTATE_FLAG": object,"FLEXIBLE_FLD_1_CHAR": object,"FLEXIBLE_FLD_2_CHAR": object}, 
                   sep="|")
#　PRC_VERS_TYP_ID is int that contain NaN, work-around is keep it as obejct
px.head()

,PATIENT_ID,CLAIM_ID,CLAIM_LINE_ITEM,CLAIM_TYP_CD,PROCEDURE_CODE,PRC1_MOD_CD,PRC1_MOD_DESC,PRC_VERS_TYP_ID,PROVIDER_BILLING_ID,PROVIDER_FACILITY_ID,...,NDC,PRODUCT,DIAGNOSIS_CODE,DIAG_CD_POSN_NBR,DIAG_VERS_TYP_ID,DIAG_DESC,WEEK_END_FRI,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR
0,999944505,10458194885000421223,4,P,86300,NaN,NaN,-1,7976755,0,...,NaN,NaN,C50.111,1,2,NaN,17-APR-20,0,1588637250,NaN
1,999944505,10458194885000421223,1,P,80053,NaN,NaN,-1,7976755,0,...,NaN,NaN,C50.111,1,2,NaN,17-APR-20,0,1588637250,NaN
2,999944505,10458181473000479341,2,P,86300,NaN,NaN,-1,7976755,0,...,NaN,NaN,C50.111,1,2,NaN,28-FEB-20,0,1588637250,NaN
3,999944505,10458181473000479341,1,P,80053,NaN,NaN,-1,7976755,0,...,NaN,NaN,C50.111,1,2,NaN,28-FEB-20,0,1588637250,NaN
4,999944505,10458165065000341912,2,P,86300,NaN,NaN,-1,7976755,0,...,NaN,NaN,C50.111,1,2,NaN,28-FEB-20,0,1588637250,NaN


In [7]:
px.count()

PATIENT_ID               9297565
CLAIM_ID                 9297565
CLAIM_LINE_ITEM          9297565
CLAIM_TYP_CD             9297565
PROCEDURE_CODE           8509820
PRC1_MOD_CD              1699860
PRC1_MOD_DESC            1698128
PRC_VERS_TYP_ID          8509820
PROVIDER_BILLING_ID      9297565
PROVIDER_FACILITY_ID     9297565
PROVIDER_REFERRING_ID    9297565
PROVIDER_RENDERING_ID    9297565
SVC_CRGD_AMT             9116032
SERVICE_DATE             9297565
MONTH_ID                 9297565
UNIT_OF_SVC_AMT          9113568
PLACE_OF_SERVICE         3513488
PAYER_PLAN_ID            8861704
PAY_TYPE                 8861704
NDC                      1421727
PRODUCT                        0
DIAGNOSIS_CODE           9297397
DIAG_CD_POSN_NBR         9297565
DIAG_VERS_TYP_ID         9297565
DIAG_DESC                      0
WEEK_END_FRI             9297565
RESTATE_FLAG             9297565
FLEXIBLE_FLD_1_CHAR      8897669
FLEXIBLE_FLD_2_CHAR            0
dtype: int64

In [8]:
rx = pd.read_csv('Data/APLD Raw Data/RX.txt/RX.txt', 
                   dtype={"PATIENT_ID": int,"CLAIM_ID": object,"NDC": object,"PROVIDER_ID": object,
                          "DIAGNOSIS_CODE": object,"DIAG_VERS_TYP_ID": object,"PAYER_PLAN_ID": object,
                          "REFILL_CODE": object,"DSPNSD_QTY": float,"DAYS_SUPPLY": int,"SERVICE_DATE": str,
                          "MONTH_ID": int,
                          "RESTATE_FLAG": object,"FLEXIBLE_FLD_1_CHAR": object,"FLEXIBLE_FLD_2_CHAR": object}, 
                   sep="|")
rx.head()

,CLAIM_ID,PATIENT_ID,NDC,PROVIDER_ID,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,PAYER_PLAN_ID,REFILL_CODE,DSPNSD_QTY,DAYS_SUPPLY,SERVICE_DATE,MONTH_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR
0,808936764247,686508685,00310072010,6872536,174.9,1,0000000000,1,10.0,28,12/09/2014,201412,0,NaN,NaN
1,808936764246,686508685,00310072010,6872536,174.9,1,0000000000,1,10.0,28,11/06/2014,201411,0,NaN,NaN
2,803520062163,78944152,00078059451,7306476,C78.7,2,0000000000,0,14.0,14,06/23/2014,201406,0,NaN,NaN
3,803519984774,78944152,00078056651,7306476,C78.7,2,0000000000,0,14.0,14,07/07/2014,201407,0,NaN,NaN
4,803519980128,78944152,00078056751,7306476,C78.7,2,0000000000,0,14.0,14,08/04/2014,201408,0,NaN,NaN


In [9]:
len(rx)

317323

**Define BC**

In [10]:
bc_sn_code = pd.read_csv('Data/BC_SN ICD CODE.csv',header=0)
bc_sn_code

,diagnosis_cd,indication_cd,version,sub_indication_cd
0,174,BC,ICD_9,NaN
1,174.0,BC,ICD_9,NaN
2,174.1,BC,ICD_9,NaN
3,174.2,BC,ICD_9,NaN
4,174.3,BC,ICD_9,NaN
...,...,...,...,...
114,C79.71,SN,ICD_10,META_STATIC
115,C79.72,SN,ICD_10,META_STATIC
116,C79.82,SN,ICD_10,META_STATIC
117,C79.89,SN,ICD_10,META_STATIC


In [11]:
bc_sn_code.rename(index=str, columns={"diagnosis_cd": "DIAGNOSIS_CODE"},inplace=True)
diag = pd.merge(diag,bc_sn_code,how='left',on='DIAGNOSIS_CODE')

In [12]:
diag.head()

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR,indication_cd,version,sub_indication_cd
0,999944505,10458194885000421223,P,12/12/2019,201912,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN
1,999944505,10458181473000479341,P,09/10/2019,201909,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN
2,999944505,10458165065000341912,P,05/20/2019,201905,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN
3,999944505,10458161262800585500,P,05/20/2019,201905,C50.111,2,1,8117971,0,1285667204,NaN,BC,ICD_10,NaN
4,999944505,10458157449200367497,P,02/21/2019,201902,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN


In [13]:
diag['SERVICE_DATE'] = pd.to_datetime(diag['SERVICE_DATE'])

In [14]:
diag_BC = diag[diag['indication_cd']=='BC']
len(diag_BC)

2036160

In [15]:
len(diag_BC['PATIENT_ID'].unique())

19911

In [16]:
diag_BC = diag_BC.loc[diag_BC.groupby('PATIENT_ID').SERVICE_DATE.idxmin()]

In [17]:
diag_BC_DATE = diag_BC[['PATIENT_ID','SERVICE_DATE','MONTH_ID']]
diag_BC_DATE.rename(index=str, columns={"SERVICE_DATE": "BC_DIAG_DATE"},inplace=True)
diag = pd.merge(diag,diag_BC_DATE,how='left',on='PATIENT_ID')
diag

/Users/guxinli/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID_x,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR,indication_cd,version,sub_indication_cd,BC_DIAG_DATE,MONTH_ID_y
0,999944505,10458194885000421223,P,2019-12-12,201912,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0
1,999944505,10458181473000479341,P,2019-09-10,201909,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0
2,999944505,10458165065000341912,P,2019-05-20,201905,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0
3,999944505,10458161262800585500,P,2019-05-20,201905,C50.111,2,1,8117971,0,1285667204,NaN,BC,ICD_10,NaN,2017-10-05,201710.0
4,999944505,10458157449200367497,P,2019-02-21,201902,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192457,1000086191,10236124904402649086,P,2018-05-12,201805,Z85.118,2,2,10262001,0,1962683979,NaN,NaN,NaN,NaN,2017-03-22,201703.0
3192458,1000086191,10236122129602285499,P,2018-05-12,201805,Z85.118,2,2,10262001,0,1962683979,NaN,NaN,NaN,NaN,2017-03-22,201703.0
3192459,1000086191,10236121853803198751,P,2018-05-06,201805,R91.8,2,1,9892518,0,1891953816,NaN,NaN,NaN,NaN,2017-03-22,201703.0
3192460,1000086191,10236090338103408273,P,2017-02-05,201702,R91.8,2,2,7849808,0,1851374607,NaN,NaN,NaN,NaN,2017-03-22,201703.0


In [18]:
diag_SN = diag[diag['indication_cd']=='SN']
len(diag_SN)

576672

In [19]:
diag_SN = diag_SN.loc[diag_SN.groupby('PATIENT_ID').SERVICE_DATE.idxmin()]
len(diag_SN)

8871

In [20]:
diag_SN_DATE= diag_SN[['PATIENT_ID','SERVICE_DATE']]
diag_SN_DATE.rename(index=str, columns={"SERVICE_DATE": "SN_DIAG_DATE"},inplace=True)
diag = pd.merge(diag,diag_SN_DATE,how='left',on='PATIENT_ID')
diag

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID_x,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR,indication_cd,version,sub_indication_cd,BC_DIAG_DATE,MONTH_ID_y,SN_DIAG_DATE
0,999944505,10458194885000421223,P,2019-12-12,201912,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT
1,999944505,10458181473000479341,P,2019-09-10,201909,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT
2,999944505,10458165065000341912,P,2019-05-20,201905,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT
3,999944505,10458161262800585500,P,2019-05-20,201905,C50.111,2,1,8117971,0,1285667204,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT
4,999944505,10458157449200367497,P,2019-02-21,201902,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192457,1000086191,10236124904402649086,P,2018-05-12,201805,Z85.118,2,2,10262001,0,1962683979,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22
3192458,1000086191,10236122129602285499,P,2018-05-12,201805,Z85.118,2,2,10262001,0,1962683979,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22
3192459,1000086191,10236121853803198751,P,2018-05-06,201805,R91.8,2,1,9892518,0,1891953816,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22
3192460,1000086191,10236090338103408273,P,2017-02-05,201702,R91.8,2,2,7849808,0,1851374607,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22


In [21]:
diag['mBC'] = np.where((diag['SN_DIAG_DATE']-diag['BC_DIAG_DATE'])>=-pd.Timedelta(30,'D'),'mBC',False) 
diag

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID_x,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR,indication_cd,version,sub_indication_cd,BC_DIAG_DATE,MONTH_ID_y,SN_DIAG_DATE,mBC
0,999944505,10458194885000421223,P,2019-12-12,201912,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False
1,999944505,10458181473000479341,P,2019-09-10,201909,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False
2,999944505,10458165065000341912,P,2019-05-20,201905,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False
3,999944505,10458161262800585500,P,2019-05-20,201905,C50.111,2,1,8117971,0,1285667204,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False
4,999944505,10458157449200367497,P,2019-02-21,201902,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192457,1000086191,10236124904402649086,P,2018-05-12,201805,Z85.118,2,2,10262001,0,1962683979,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC
3192458,1000086191,10236122129602285499,P,2018-05-12,201805,Z85.118,2,2,10262001,0,1962683979,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC
3192459,1000086191,10236121853803198751,P,2018-05-06,201805,R91.8,2,1,9892518,0,1891953816,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC
3192460,1000086191,10236090338103408273,P,2017-02-05,201702,R91.8,2,2,7849808,0,1851374607,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC


In [22]:
diag['DIAG'] = np.where(((diag['BC_DIAG_DATE'].notnull() & diag['SN_DIAG_DATE'].isnull()) | ((diag['SN_DIAG_DATE']-diag['BC_DIAG_DATE'])<-pd.Timedelta(30,'D'))),'BC',diag['mBC'])
diag

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID_x,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,PROVIDER_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR,indication_cd,version,sub_indication_cd,BC_DIAG_DATE,MONTH_ID_y,SN_DIAG_DATE,mBC,DIAG
0,999944505,10458194885000421223,P,2019-12-12,201912,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False,BC
1,999944505,10458181473000479341,P,2019-09-10,201909,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False,BC
2,999944505,10458165065000341912,P,2019-05-20,201905,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False,BC
3,999944505,10458161262800585500,P,2019-05-20,201905,C50.111,2,1,8117971,0,1285667204,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False,BC
4,999944505,10458157449200367497,P,2019-02-21,201902,C50.111,2,1,7976755,0,1588637250,NaN,BC,ICD_10,NaN,2017-10-05,201710.0,NaT,False,BC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192457,1000086191,10236124904402649086,P,2018-05-12,201805,Z85.118,2,2,10262001,0,1962683979,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC,mBC
3192458,1000086191,10236122129602285499,P,2018-05-12,201805,Z85.118,2,2,10262001,0,1962683979,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC,mBC
3192459,1000086191,10236121853803198751,P,2018-05-06,201805,R91.8,2,1,9892518,0,1891953816,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC,mBC
3192460,1000086191,10236090338103408273,P,2017-02-05,201702,R91.8,2,2,7849808,0,1851374607,NaN,NaN,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC,mBC


In [174]:
diag.to_csv("merged_diag_with_label.csv", index= False)   

In [176]:
diag.columns

Index(['PATIENT_ID', 'CLAIM_ID', 'CLAIM_TYP_CD', 'SERVICE_DATE', 'MONTH_ID_x',
       'DIAGNOSIS_CODE', 'DIAG_VERS_TYP_ID', 'DIAG_CD_POSN_NBR', 'PROVIDER_ID',
       'RESTATE_FLAG', 'FLEXIBLE_FLD_1_CHAR', 'FLEXIBLE_FLD_2_CHAR',
       'indication_cd', 'version', 'sub_indication_cd', 'BC_DIAG_DATE',
       'MONTH_ID_y', 'SN_DIAG_DATE', 'mBC', 'DIAG'],
      dtype='object')

In [23]:
diag_selected = diag[['PATIENT_ID','CLAIM_ID','CLAIM_TYP_CD','SERVICE_DATE', 'MONTH_ID_x','DIAGNOSIS_CODE', 'DIAG_VERS_TYP_ID',
                     'DIAG_CD_POSN_NBR','RESTATE_FLAG','indication_cd','sub_indication_cd', 'BC_DIAG_DATE',
                      'MONTH_ID_y', 'SN_DIAG_DATE', 'mBC', 'DIAG']]
diag_selected

,PATIENT_ID,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE,MONTH_ID_x,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,RESTATE_FLAG,indication_cd,sub_indication_cd,BC_DIAG_DATE,MONTH_ID_y,SN_DIAG_DATE,mBC,DIAG
0,999944505,10458194885000421223,P,2019-12-12,201912,C50.111,2,1,0,BC,NaN,2017-10-05,201710.0,NaT,False,BC
1,999944505,10458181473000479341,P,2019-09-10,201909,C50.111,2,1,0,BC,NaN,2017-10-05,201710.0,NaT,False,BC
2,999944505,10458165065000341912,P,2019-05-20,201905,C50.111,2,1,0,BC,NaN,2017-10-05,201710.0,NaT,False,BC
3,999944505,10458161262800585500,P,2019-05-20,201905,C50.111,2,1,0,BC,NaN,2017-10-05,201710.0,NaT,False,BC
4,999944505,10458157449200367497,P,2019-02-21,201902,C50.111,2,1,0,BC,NaN,2017-10-05,201710.0,NaT,False,BC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3192457,1000086191,10236124904402649086,P,2018-05-12,201805,Z85.118,2,2,0,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC,mBC
3192458,1000086191,10236122129602285499,P,2018-05-12,201805,Z85.118,2,2,0,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC,mBC
3192459,1000086191,10236121853803198751,P,2018-05-06,201805,R91.8,2,1,0,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC,mBC
3192460,1000086191,10236090338103408273,P,2017-02-05,201702,R91.8,2,2,0,NaN,NaN,2017-03-22,201703.0,2017-03-22,mBC,mBC


In [31]:
df1 = pd.DataFrame(px['PATIENT_ID'].unique())
df2 = pd.DataFrame(diag['PATIENT_ID'].unique())
df3 = pd.DataFrame(rx['PATIENT_ID'].unique())
intersected_df = pd.merge(df1, df2, how='inner')
len(intersected_df)

20000

In [32]:
len(df3)

17188

**COMBINE px&rx, define drugs**

In [184]:
len(px['PROCEDURE_CODE'].unique())

6619

In [185]:
len(rx['NDC'].unique())

1187

In [24]:
dg_rf = pd.read_csv('Data/APLD Raw Data/PRODUCT_reference_20200925.csv',header=0)
dg_rf.head()

,NDC,GPI_DESC,DRUG_NAME,BGI,DRUG_GENERIC_NAME,DRUG_STRENGTH,DRUG_FORM,PACK_QTY,PACKAGE_DESCRIPTION,MANUFACTURER,USC_CD,USC_DESC,NDC_START_DATE,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR
0,99999099211,*COVID-19 Test Specimen Collection Kit***,COVID-19 TEST SPECIMEN CO,G,COVID-19 TEST SPECIMEN COLLECTION,NaN,KIT,1.0,PACKAGE,NCPDP EMERGENCY PREPAREDNESS,40400.0,"DIAGNOSTIC AIDS, OTHER",06/22/2020,NaN,NaN
1,99970001014,NaN,ACNEWORX,U,NaN,2%,NaN,NaN,NaN,DERMWORX INC,37110.0,ACNE W/O ANTNFCT/ANTISEPTIC,NaN,NaN,NaN
2,99866027824,*Misc Natural Products Tab**,FLEX-A-MIN TRIPLE STRENGT,T,MISC NATURAL PRODUCTS,NaN,TABS,1.0,BOTTLE,NATURES BOUNTY,84290.0,"NATURAL MED OTHER, OTHER",07/15/2013,NaN,NaN
3,99866027823,*Misc Natural Products Tab**,FLEX-A-MIN TRIPLE STRENGT,T,MISC NATURAL PRODUCTS,NaN,TABS,1.0,BOTTLE,NATURES BOUNTY,84290.0,"NATURAL MED OTHER, OTHER",07/15/2013,NaN,NaN
4,99866027822,*Misc Natural Products Tab**,FLEX-A-MIN TRIPLE STRENGT,T,MISC NATURAL PRODUCTS,NaN,TABS,1.0,BOTTLE,NATURES BOUNTY,84290.0,"NATURAL MED OTHER, OTHER",07/15/2013,NaN,NaN


In [25]:
dg_rf.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532514 entries, 0 to 532513
Data columns (total 15 columns):
NDC                    532514 non-null int64
GPI_DESC               511578 non-null object
DRUG_NAME              526671 non-null object
BGI                    532514 non-null object
DRUG_GENERIC_NAME      509084 non-null object
DRUG_STRENGTH          330240 non-null object
DRUG_FORM              427758 non-null object
PACK_QTY               510669 non-null float64
PACKAGE_DESCRIPTION    508939 non-null object
MANUFACTURER           521506 non-null object
USC_CD                 528266 non-null float64
USC_DESC               527357 non-null object
NDC_START_DATE         499423 non-null object
FLEXIBLE_FLD_1_CHAR    0 non-null float64
FLEXIBLE_FLD_2_CHAR    0 non-null float64
dtypes: float64(4), int64(1), object(10)
memory usage: 60.9+ MB


In [54]:
px.columns

Index(['PATIENT_ID', 'CLAIM_ID', 'CLAIM_LINE_ITEM', 'CLAIM_TYP_CD',
       'PROCEDURE_CODE', 'PRC1_MOD_CD', 'PRC1_MOD_DESC', 'PRC_VERS_TYP_ID',
       'PROVIDER_BILLING_ID', 'PROVIDER_FACILITY_ID', 'PROVIDER_REFERRING_ID',
       'PROVIDER_RENDERING_ID', 'SVC_CRGD_AMT', 'SERVICE_DATE', 'MONTH_ID',
       'UNIT_OF_SVC_AMT', 'PLACE_OF_SERVICE', 'PAYER_PLAN_ID', 'PAY_TYPE',
       'NDC', 'PRODUCT', 'DIAGNOSIS_CODE', 'DIAG_CD_POSN_NBR',
       'DIAG_VERS_TYP_ID', 'DIAG_DESC', 'WEEK_END_FRI', 'RESTATE_FLAG',
       'FLEXIBLE_FLD_1_CHAR', 'FLEXIBLE_FLD_2_CHAR'],
      dtype='object')

In [56]:
rx.columns

Index(['CLAIM_ID', 'PATIENT_ID', 'NDC', 'PROVIDER_ID', 'DIAGNOSIS_CODE',
       'DIAG_VERS_TYP_ID', 'PAYER_PLAN_ID', 'REFILL_CODE', 'DSPNSD_QTY',
       'DAYS_SUPPLY', 'SERVICE_DATE', 'MONTH_ID', 'RESTATE_FLAG',
       'FLEXIBLE_FLD_1_CHAR', 'FLEXIBLE_FLD_2_CHAR'],
      dtype='object')

In [57]:
rx.head()

,CLAIM_ID,PATIENT_ID,NDC,PROVIDER_ID,DIAGNOSIS_CODE,DIAG_VERS_TYP_ID,PAYER_PLAN_ID,REFILL_CODE,DSPNSD_QTY,DAYS_SUPPLY,SERVICE_DATE,MONTH_ID,RESTATE_FLAG,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR
0,808936764247,686508685,00310072010,6872536,174.9,1,0000000000,1,10.0,28,12/09/2014,201412,0,NaN,NaN
1,808936764246,686508685,00310072010,6872536,174.9,1,0000000000,1,10.0,28,11/06/2014,201411,0,NaN,NaN
2,803520062163,78944152,00078059451,7306476,C78.7,2,0000000000,0,14.0,14,06/23/2014,201406,0,NaN,NaN
3,803519984774,78944152,00078056651,7306476,C78.7,2,0000000000,0,14.0,14,07/07/2014,201407,0,NaN,NaN
4,803519980128,78944152,00078056751,7306476,C78.7,2,0000000000,0,14.0,14,08/04/2014,201408,0,NaN,NaN


In [72]:
import sqlite3
from sqlalchemy import *
engine = create_engine('sqlite:///Novartis.sqlite', echo=False)
dg_rf.to_sql('dg_rf', con=engine, if_exists='replace', index=False)
engine.execute("SELECT * FROM dg_rf").fetchall()

[(99999099211, '*COVID-19 Test Specimen Collection Kit***', 'COVID-19 TEST SPECIMEN CO', 'G', 'COVID-19 TEST SPECIMEN COLLECTION', None, 'KIT', 1.0, 'PACKAGE', 'NCPDP EMERGENCY PREPAREDNESS', 40400.0, 'DIAGNOSTIC AIDS, OTHER', '06/22/2020', None, None),
 (99970001014, None, 'ACNEWORX', 'U', None, '2%', None, None, None, 'DERMWORX INC', 37110.0, 'ACNE W/O ANTNFCT/ANTISEPTIC', None, None, None),
 (99866027824, '*Misc Natural Products Tab**', 'FLEX-A-MIN TRIPLE STRENGT', 'T', 'MISC NATURAL PRODUCTS', None, 'TABS', 1.0, 'BOTTLE', 'NATURES BOUNTY', 84290.0, 'NATURAL MED OTHER, OTHER', '07/15/2013', None, None),
 (99866027823, '*Misc Natural Products Tab**', 'FLEX-A-MIN TRIPLE STRENGT', 'T', 'MISC NATURAL PRODUCTS', None, 'TABS', 1.0, 'BOTTLE', 'NATURES BOUNTY', 84290.0, 'NATURAL MED OTHER, OTHER', '07/15/2013', None, None),
 (99866027822, '*Misc Natural Products Tab**', 'FLEX-A-MIN TRIPLE STRENGT', 'T', 'MISC NATURAL PRODUCTS', None, 'TABS', 1.0, 'BOTTLE', 'NATURES BOUNTY', 84290.0, 'NATURA

In [27]:
conn = sqlite3.connect('Novartis.sqlite')
cur = conn.cursor()
dg_rf_new = cur.execute('''
SELECT *,
CASE 
WHEN  ((DRUG_GENERIC_NAME LIKE '%PALBOCICLIB%') OR (DRUG_NAME LIKE '%IBRANCE%')) THEN 'IBR' 
WHEN  ((DRUG_GENERIC_NAME LIKE '%FULVESTRANT%') OR (DRUG_NAME LIKE '%FASLODEX%')) THEN 'FAS' 
WHEN  ((DRUG_GENERIC_NAME LIKE '%RIBOCICLIB%') OR (DRUG_NAME LIKE '%KISQALI%') OR (DRUG_NAME LIKE '%KISQALI FEMARA%') OR (DRUG_GENERIC_NAME LIKE '%RIBOCICLIB SUCCINATE-LETROZOLE%')) THEN 'KIS' 
WHEN  ((DRUG_GENERIC_NAME LIKE '%EVEROLIMUS%') OR (DRUG_NAME LIKE '%AFINITOR%')) THEN 'AFI' 
WHEN  ((DRUG_GENERIC_NAME LIKE '%LETROZOLE%') OR (DRUG_NAME LIKE '%FEMARA%')) THEN 'LET' 
WHEN  ((DRUG_GENERIC_NAME LIKE '%ABEMACICLIB%') OR (DRUG_NAME LIKE '%VERZENIO%')) THEN 'VER' 
WHEN  ((DRUG_GENERIC_NAME LIKE '%CHEMO%') OR (DRUG_NAME LIKE '%CHEMO%')) THEN 'CHEMO' 
WHEN  ((DRUG_GENERIC_NAME LIKE '%CAPECITABINE%') OR (DRUG_NAME LIKE '%XELODA%')) THEN 'XEL' 
WHEN  ((DRUG_NAME LIKE '%TAMOXIFEN CITRATE%')) THEN 'TAM' 
WHEN  ((DRUG_GENERIC_NAME LIKE '%ANASTROZOLE%') OR (DRUG_GENERIC_NAME LIKE '%EXEMESTANE%') OR (DRUG_NAME LIKE '%ANASTROZOLE%') OR (DRUG_NAME LIKE '%ARIMIDEX%') OR (DRUG_NAME LIKE '%EXEMESTANE%') OR (DRUG_NAME LIKE '%AROMASIN%' ) OR (DRUG_NAME LIKE '%NOLVADEX%') OR (DRUG_NAME LIKE '%SOLTAMOX%')) THEN 'AI' 
ELSE 'OTHERS' 
END AS brand 
FROM dg_rf
''').fetchall()

In [28]:
dg_rf.columns

Index(['NDC', 'GPI_DESC', 'DRUG_NAME', 'BGI', 'DRUG_GENERIC_NAME',
       'DRUG_STRENGTH', 'DRUG_FORM', 'PACK_QTY', 'PACKAGE_DESCRIPTION',
       'MANUFACTURER', 'USC_CD', 'USC_DESC', 'NDC_START_DATE',
       'FLEXIBLE_FLD_1_CHAR', 'FLEXIBLE_FLD_2_CHAR'],
      dtype='object')

In [29]:
dg_rf_new = pd.DataFrame(dg_rf_new,columns=['NDC', 'GPI_DESC', 'DRUG_NAME', 'BGI', 'DRUG_GENERIC_NAME',
       'DRUG_STRENGTH', 'DRUG_FORM', 'PACK_QTY', 'PACKAGE_DESCRIPTION',
       'MANUFACTURER', 'USC_CD', 'USC_DESC', 'NDC_START_DATE',
       'FLEXIBLE_FLD_1_CHAR', 'FLEXIBLE_FLD_2_CHAR','BRAND'])
dg_rf_new

,NDC,GPI_DESC,DRUG_NAME,BGI,DRUG_GENERIC_NAME,DRUG_STRENGTH,DRUG_FORM,PACK_QTY,PACKAGE_DESCRIPTION,MANUFACTURER,USC_CD,USC_DESC,NDC_START_DATE,FLEXIBLE_FLD_1_CHAR,FLEXIBLE_FLD_2_CHAR,BRAND
0,99999099211,*COVID-19 Test Specimen Collection Kit***,COVID-19 TEST SPECIMEN CO,G,COVID-19 TEST SPECIMEN COLLECTION,None,KIT,1.0,PACKAGE,NCPDP EMERGENCY PREPAREDNESS,40400.0,"DIAGNOSTIC AIDS, OTHER",06/22/2020,None,None,OTHERS
1,99970001014,None,ACNEWORX,U,None,2%,None,NaN,None,DERMWORX INC,37110.0,ACNE W/O ANTNFCT/ANTISEPTIC,None,None,None,OTHERS
2,99866027824,*Misc Natural Products Tab**,FLEX-A-MIN TRIPLE STRENGT,T,MISC NATURAL PRODUCTS,None,TABS,1.0,BOTTLE,NATURES BOUNTY,84290.0,"NATURAL MED OTHER, OTHER",07/15/2013,None,None,OTHERS
3,99866027823,*Misc Natural Products Tab**,FLEX-A-MIN TRIPLE STRENGT,T,MISC NATURAL PRODUCTS,None,TABS,1.0,BOTTLE,NATURES BOUNTY,84290.0,"NATURAL MED OTHER, OTHER",07/15/2013,None,None,OTHERS
4,99866027822,*Misc Natural Products Tab**,FLEX-A-MIN TRIPLE STRENGT,T,MISC NATURAL PRODUCTS,None,TABS,1.0,BOTTLE,NATURES BOUNTY,84290.0,"NATURAL MED OTHER, OTHER",07/15/2013,None,None,OTHERS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532509,1296640,None,RAPTIVA,T,None,125MG,None,NaN,None,"GENENTECH,INC.",86230.0,MONOCLONAL ANTIBODIES,None,None,None,OTHERS
532510,1224333,None,AUSTRALIAN DREAM,U,None,None,None,NaN,None,NATURES HEALTH CON,84290.0,"NATURAL MED OTHER, OTHER",None,None,None,OTHERS
532511,1003298,None,BIO ST JOHNS,U,None,None,None,NaN,None,"PHARMANEX, INC.",84210.0,"NATURAL MED OTHER, HERBALS",None,None,None,OTHERS
532512,160001,None,SLEEPINAL,U,None,50MG,None,NaN,None,THOMPSON MEDICAL,67290.0,"NON-BARB, OTHER",None,None,None,OTHERS


In [30]:
dg_rf_new = dg_rf_new[dg_rf_new['BRAND']!='OTHERS']

**Combine PX & RX**

In [31]:
rx_fil = rx.loc[:, ["PATIENT_ID", "CLAIM_ID", "SERVICE_DATE","MONTH_ID", "NDC", "DIAGNOSIS_CODE"]]
px_fil = px.loc[:, ["PATIENT_ID", "CLAIM_ID",  "SERVICE_DATE","MONTH_ID", "PROCEDURE_CODE", "DIAGNOSIS_CODE"]]

In [32]:
rx_fil = rx_fil.rename(columns={"NDC":"drug_id"})
px_fil = px_fil.rename(columns={"PROCEDURE_CODE":"drug_id",})
# rx_fil.shape (317323, 6)
# px_fil.shape (9297565, 6)

In [33]:
px_fil_nodup = px_fil.drop_duplicates()
#px_fil_nodup.shape (8613515, 6), indicates duplicates rows, this px dataframe will be further used for merge with rx
rx_fil_nodup = rx_fil.drop_duplicates()
#rx_fil_nodup.shape (317322, 6), indicates all the records are unique

In [34]:
merged = rx_fil_nodup.append(px_fil_nodup, ignore_index=True)

In [204]:
merged

,PATIENT_ID,CLAIM_ID,SERVICE_DATE,MONTH_ID,drug_id,DIAGNOSIS_CODE
0,686508685,808936764247,12/09/2014,201412,00310072010,174.9
1,686508685,808936764246,11/06/2014,201411,00310072010,174.9
2,78944152,803520062163,06/23/2014,201406,00078059451,C78.7
3,78944152,803519984774,07/07/2014,201407,00078056651,C78.7
4,78944152,803519980128,08/04/2014,201408,00078056751,C78.7
...,...,...,...,...,...,...
8930832,1000086191,10236122129602285499,05/12/2018,201805,70553,Z85.118
8930833,1000086191,10236122129602285499,05/12/2018,201805,70544,Z85.118
8930834,1000086191,10236121853803198751,05/06/2018,201805,71045,R91.8
8930835,1000086191,10236090338103408273,02/05/2017,201702,71010,R91.8


In [35]:
merged = merged.sort_values(by="PATIENT_ID", ascending=False) 
merged_nodup = merged.drop_duplicates()

In [207]:
merged_nodup

,PATIENT_ID,CLAIM_ID,SERVICE_DATE,MONTH_ID,drug_id,DIAGNOSIS_CODE
5385720,1734677597,127836631746,10/15/2015,201510,99213,C50.411
295799,1734677597,10040133709200031466,09/10/2018,201809,00093762056,NaN
5385831,1734677597,10655089460200320788,12/30/2014,201412,38525,174.4
5385832,1734677597,10655089460200320788,12/30/2014,201412,19301,174.4
296577,1734677597,10040111477300027445,02/07/2018,201802,51991075933,NaN
...,...,...,...,...,...,...
4873206,29388541,10364098560001801207,06/06/2017,201706,J9267,Z51.11
4873205,29388541,10364098560001801207,06/06/2017,201706,J9267,Z51.12
4873204,29388541,10364098560001801207,06/06/2017,201706,J9355,C50.012
4873203,29388541,10364098560001801207,06/06/2017,201706,J9355,Z17.0


In [36]:
merged_nodup = merged_nodup.astype({'drug_id':'str'})

In [37]:
merged_nodup.head()

,PATIENT_ID,CLAIM_ID,SERVICE_DATE,MONTH_ID,drug_id,DIAGNOSIS_CODE
5385720,1734677597,127836631746,10/15/2015,201510,99213,C50.411
295799,1734677597,10040133709200031466,09/10/2018,201809,00093762056,NaN
5385831,1734677597,10655089460200320788,12/30/2014,201412,38525,174.4
5385832,1734677597,10655089460200320788,12/30/2014,201412,19301,174.4
296577,1734677597,10040111477300027445,02/07/2018,201802,51991075933,NaN


In [38]:
dg_rf_new.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 555 entries, 5077 to 531930
Data columns (total 16 columns):
NDC                    555 non-null int64
GPI_DESC               555 non-null object
DRUG_NAME              555 non-null object
BGI                    555 non-null object
DRUG_GENERIC_NAME      551 non-null object
DRUG_STRENGTH          391 non-null object
DRUG_FORM              525 non-null object
PACK_QTY               555 non-null float64
PACKAGE_DESCRIPTION    555 non-null object
MANUFACTURER           555 non-null object
USC_CD                 555 non-null float64
USC_DESC               555 non-null object
NDC_START_DATE         553 non-null object
FLEXIBLE_FLD_1_CHAR    0 non-null object
FLEXIBLE_FLD_2_CHAR    0 non-null object
BRAND                  555 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 73.7+ KB


In [39]:
dg_rf_new=dg_rf_new.astype({'NDC': 'str'})

In [40]:
dg_rf_new = dg_rf_new[['NDC','DRUG_NAME','DRUG_GENERIC_NAME','BRAND']]

In [41]:
rx_px_drug = pd.merge(merged_nodup,dg_rf_new,how='left',left_on='drug_id',right_on='NDC')
rx_px_drug.head()

,PATIENT_ID,CLAIM_ID,SERVICE_DATE,MONTH_ID,drug_id,DIAGNOSIS_CODE,NDC,DRUG_NAME,DRUG_GENERIC_NAME,BRAND
0,1734677597,127836631746,10/15/2015,201510,99213,C50.411,NaN,NaN,NaN,NaN
1,1734677597,10040133709200031466,09/10/2018,201809,00093762056,NaN,NaN,NaN,NaN,NaN
2,1734677597,10655089460200320788,12/30/2014,201412,38525,174.4,NaN,NaN,NaN,NaN
3,1734677597,10655089460200320788,12/30/2014,201412,19301,174.4,NaN,NaN,NaN,NaN
4,1734677597,10040111477300027445,02/07/2018,201802,51991075933,NaN,51991075933,LETROZOLE,LETROZOLE,LET


In [44]:
rx_px_diag = pd.merge(diag_selected,rx_px_drug,how='outer',on='CLAIM_ID')

In [45]:
rx_px_diag.head()

,PATIENT_ID_x,CLAIM_ID,CLAIM_TYP_CD,SERVICE_DATE_x,MONTH_ID_x,DIAGNOSIS_CODE_x,DIAG_VERS_TYP_ID,DIAG_CD_POSN_NBR,RESTATE_FLAG,indication_cd,...,DIAG,PATIENT_ID_y,SERVICE_DATE_y,MONTH_ID,drug_id,DIAGNOSIS_CODE_y,NDC,DRUG_NAME,DRUG_GENERIC_NAME,BRAND
0,999944505.0,10458194885000421223,P,2019-12-12,201912.0,C50.111,2.0,1.0,0,BC,...,BC,999944505.0,12/12/2019,201912.0,86300,C50.111,NaN,NaN,NaN,NaN
1,999944505.0,10458194885000421223,P,2019-12-12,201912.0,C50.111,2.0,1.0,0,BC,...,BC,999944505.0,12/12/2019,201912.0,80053,C50.111,NaN,NaN,NaN,NaN
2,999944505.0,10458181473000479341,P,2019-09-10,201909.0,C50.111,2.0,1.0,0,BC,...,BC,999944505.0,09/10/2019,201909.0,86300,C50.111,NaN,NaN,NaN,NaN
3,999944505.0,10458181473000479341,P,2019-09-10,201909.0,C50.111,2.0,1.0,0,BC,...,BC,999944505.0,09/10/2019,201909.0,80053,C50.111,NaN,NaN,NaN,NaN
4,999944505.0,10458165065000341912,P,2019-05-20,201905.0,C50.111,2.0,1.0,0,BC,...,BC,999944505.0,05/20/2019,201905.0,86300,C50.111,NaN,NaN,NaN,NaN


In [46]:
len(rx_px_diag)

48519252

In [ ]:
len(rx_px_diag[rx_px_diag['NDC'].notnull()])